In [2]:
import pandas as pd
import requests
import json
from pprint import pprint
from config import openWeatherKey

ModuleNotFoundError: No module named 'config'

In [3]:
#Pulling data from csv files and creating dataframe with disaster data
coordinates_csv = "output_data/NEW_coordinates.csv"
weatherDisasterCSV = "Resources/disaster_data.csv"
wDisaster_df = pd.read_csv(weatherDisasterCSV)
desiredColumns = ["Dis No", "Year", "Seq", "Disaster Subgroup", "Disaster Type", "Disaster Subtype", "Disaster Subsubtype", "Entry Criteria", "ISO", "Location", "Start Month", "End Month", "Total Deaths", "No Injured", "No Affected", "No Homeless", "Total Affected", "Insured Damages ('000 US$)", "Total Damages ('000 US$)", "CPI"]
wDisaster_df = wDisaster_df.loc[wDisaster_df["ISO"] == "USA", desiredColumns]
wDisaster_df

,Dis No,Year,Seq,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Entry Criteria,ISO,Location,Start Month,End Month,Total Deaths,No Injured,No Affected,No Homeless,Total Affected,Insured Damages ('000 US$),Total Damages ('000 US$),CPI
2,2000-0080-USA,2000,80,Hydrological,Flood,Riverine flood,NaN,Affect,USA,"Kentucky, Ohio provinces",2,2.0,3.0,NaN,231.0,NaN,231.0,NaN,NaN,67.355759
3,2000-0500-USA,2000,500,Hydrological,Flood,Riverine flood,NaN,Affect,USA,"Morris, Sussex districts (New Jersey province)",8,8.0,NaN,NaN,175.0,NaN,175.0,NaN,166000.0,67.355759
4,2000-0919-USA,2000,919,Meteorological,Storm,NaN,NaN,SigDam,USA,"Alabama, Georgia, Louisiana, North Carolina, S...",1,1.0,4.0,NaN,NaN,NaN,NaN,280000.0,350000.0,67.355759
5,2000-9712-USA,2000,9712,Climatological,Drought,Drought,NaN,Affected,USA,Wyoming province,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.355759
6,2000-0838-USA,2000,838,Meteorological,Storm,Convective storm,Winter storm/Blizzard,Kill,USA,"Texas, Oklahoma, New Mexico, Arkansas, Missour...",12,12.0,57.0,34.0,NaN,NaN,34.0,200500.0,10000.0,67.355759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,2019-0307-USA,2019,307,Hydrological,Flood,NaN,NaN,Affected,USA,"Lincolnton region (Lincoln County, North Carol...",6,6.0,3.0,NaN,600.0,NaN,600.0,NaN,NaN,100.000000
533,2020-0148-USA,2020,148,Meteorological,Storm,Convective storm,Tornado,Kill,USA,"Louisiana, Texas, Mississippi, South Carolina,...",4,4.0,38.0,200.0,NaN,NaN,200.0,NaN,1000000.0,NaN
534,2020-0011-USA,2020,11,Meteorological,Storm,Convective storm,Severe storm,Kill,USA,"Texas, Oklahoma, Missouri, Arkansas, Louisiana...",1,1.0,12.0,NaN,NaN,NaN,NaN,NaN,1200000.0,NaN
535,2020-0082-USA,2020,82,Meteorological,Storm,Convective storm,Tornado,Waiting,USA,"Nashville (Tennessee), Kentucky, Missouri, Mis...",3,3.0,25.0,300.0,12000.0,NaN,12300.0,NaN,1100000.0,NaN


In [4]:
#Breaksdown and counts all unique locations
#wDisaster_df["Location"].value_counts()

In [5]:
#Creating new dataframe by splitting data into Location and Dis. No.
newData = {
    "Location": [],
    "Dis No": []
}

for index, row in wDisaster_df.iterrows():
    #print(type(newData["Location"]))
    if type(row["Location"]) is str:
        for location in row["Location"].split(","):
            newData["Location"].append(location)
            newData["Dis No"].append(row["Dis No"])
splitLocations_df = pd.DataFrame(newData)
#splitLocations_df

In [6]:
#Grouping disaster dataframe and counting based on disaster type
#wDisaster_df.groupby("Disaster Type").count()

In [7]:
#Reading coordinates csv file and creating new dataframe
coordinates_location_df = pd.read_csv(coordinates_csv)
clean_coordinates_index = coordinates_location_df.index
coordinates_location_df = coordinates_location_df.loc[:, ["Location", "Dis No", "Lat", "Long"]]
coordinates_location_df

,Location,Dis No,Lat,Long
0,Kentucky,2000-0080-USA,34.61,-92.66
1,Morris,2000-0500-USA,40.83,-74.53
2,Alabama,2000-0919-USA,32.75,-86.75
3,Georgia,2000-0919-USA,32.75,-83.50
4,Louisiana,2000-0919-USA,31.00,-92.00
...,...,...,...,...
2856,Kentucky,2020-0082-USA,34.61,-92.66
2857,Missouri,2020-0082-USA,38.25,-92.50
2858,Mississippi,2020-0082-USA,32.75,-89.75
2859,Georgia,2020-0082-USA,32.75,-83.50


In [8]:
#Northwest coordinates >40N, >110W, and Hawaii
#I need to first create a dataframe based on Longitude, <-130W, >-110W
fourth_quintant_df = coordinates_location_df.loc[(coordinates_location_df["Long"] > -130) & (coordinates_location_df["Long"] < -110), :]
#Create dataframe based on Latitude conditions >40N
fourth_quintant_n_df = fourth_quintant_df.loc[fourth_quintant_df["Lat"] > 40, :].reset_index(drop=True)
#Create dataframe based on Latitude conditions <50N
fourth_quintant_s_df = fourth_quintant_df.loc[fourth_quintant_df["Lat"] < 50, :].reset_index(drop=True)
#Create dataframes based on unique locations within the set conditions for the NW
affected_area_n_df = len(fourth_quintant_n_df["Location"].unique())
affected_area_s_df = len(fourth_quintant_s_df["Location"].unique())

In [9]:
#affected_area_n_df

In [10]:
#affected_area_s_df

In [14]:
#I need to merge lat dataframes along Location
combined_quintant_lat_df = pd.merge(fourth_quintant_n_df, fourth_quintant_s_df, on = "Dis No", how="outer")
combined_quintant_lat_df
#I need to merge long dataframe with new combined lat dataframe
# combined_quintant_nw_df = pd.merge(combined_quintant_lat_df, fourth_quintant_df, on = "Location", how="inner")
# combined_quintant_nw_df

,Location_x,Dis No,Lat_x,Long_x,Location_y,Lat_y,Long_y
0,Richland,2000-0338-USA,46.29,-119.28,Richland,46.29,-119.28
1,Idaho,2000-0465-USA,44.50,-114.25,Arizona,34.50,-111.50
2,Idaho,2000-0465-USA,44.50,-114.25,Idaho,44.50,-114.25
3,Idaho,2000-0465-USA,44.50,-114.25,Nevada,39.25,-116.75
4,Idaho,2000-0465-USA,44.50,-114.25,Oregon,44.00,-120.50
...,...,...,...,...,...,...,...
271,NaN,2018-0468-USA,NaN,NaN,Hidden Hills,34.16,-118.65
272,NaN,2019-0086-USA,NaN,NaN,Sonoma,38.29,-122.46
273,NaN,2019-0517-USA,NaN,NaN,Los Angeles,34.05,-118.24
274,NaN,2019-0517-USA,NaN,NaN,San Bernardino,34.11,-117.29


In [12]:
# #I need to create a new dataframe combining the new combine quintant coordinates and the weather disaster data
# combined_data_nw_df = pd.merge(combined_quintant_nw_df, wDisaster_df, on = "Dis No", how="outer")
# combined_data_nw_df
# #I need to recreate dataframe by dropping all NaN values
# combined_data_nw_df.replace({'Location_x': 'NaN'}, {'Location_x': float('NaN')})
# combined_data_nw_df.replace({'Total Deaths': 'NaN'}, {'Total Deaths': float('NaN')})
# combined_data_nw_df.replace({'Total Injured': 'NaN'}, {'No Injured': float('NaN')})
# clean_combined_nw_df = combined_data_nw_df.dropna(subset=["Location_x", "Total Deaths", "No Injured"])
# clean_combined_nw_df.head()


In [13]:
#Combine "Location_x" and rename "Location"
group_nw_df = clean_combined_nw_df.groupby(["Location_x", "Lat", "Long"]).agg
group_nw_df

NameError: name 'clean_combined_nw_df' is not defined

In [95]:
#I need to clean up dataframe where the Location column has no information
#clean_ = combined_data_nw_df.iloc[:,0]
# clean_loc.dropna()
# clean_loc
# clean_data_nw_loc_df = pd.merge(combined_data_nw_df, clean_loc)
# clean_data_nw_loc_df

In [96]:
#Creating a trimmed DF of dropped duplicates for more accurate calculations
unique_dis_nw_df = clean_combined_nw_df.drop_duplicates(subset="Dis No", keep="first")
#unique_dis_nw_df
unique_quintant_n = unique_dis_nw_df.loc[unique_dis_nw_df["Lat"] > 40, :].reset_index(drop=True)
#unique_quintant_n
unique_quintant_s = unique_dis_nw_df.loc[unique_dis_nw_df["Lat"] < 50, :].reset_index(drop=True)
unique_quintant_s
unique_affected_area_n = len(unique_quintant_n["Location_x"].unique())
unique_affected_area_s = len(unique_quintant_s["Location_x"].unique())
unique_affected_area_n

AttributeError: 'DataFrame' object has no attribute 'unique'

In [97]:
# fourth_quintant = combined_data_df.loc[(coordinates_location_df["Long"] < -90) & (combined_data_df["Long"] > -110), :]
# fourth_quintant_n = fourth_quintant.loc[fourth_quintant["Lat"] > 40, :].reset_index(drop=True)
# fourth_quintant_s = fourth_quintant.loc[fourth_quintant["Lat"] < 40, :].reset_index(drop=True)
# affected_area_n = len(fourth_quintant_n["Location_x"].unique())
# affected_area_s = len(fourth_quintant_s["Location_x"].unique())

In [98]:
#Create df with dropped duplicates types
#dis_no_unique_df = fourth_quintant
disaster_type = combined_data_nw_df["Disaster Type"].unique()
# disaster_type_count_n = fourth_quintant_n["Disaster Type"].value_counts()
# disaster_type_count_s = fourth_quintant_s["Disaster Type"].value_counts()
disaster_type

array(['Storm', 'Wildfire', 'Flood', 'Extreme temperature', 'Drought',
       'Earthquake', 'Epidemic', 'Landslide', 'Volcanic activity'],
      dtype=object)

In [2]:
fourth_quintant_n.loc[fourth_quintant_n["Disaster Type"] == "Storm", :]

NameError: name 'fourth_quintant_n' is not defined

In [3]:
#coordinates_location_df["Location"].value_counts()

In [58]:
# coordinates_data = {
#     "Location": [],
#     "Dis No": [],
#     "Lat": [],
#     "Long": []
# }

# for index, row in coordinates_location_df.iterrows():
#     #print(type(coordinates_data["Location"]))
#     if type(row["Location"]) is str:
#         for location in row["Location"].split(","):
#             newData["Location"].append(location)
#             newData["Dis No"].append(row["Dis No"])
# split_coordinates_df = pd.DataFrame(coordinates_data)
# split_coordinates_df

SyntaxError: invalid syntax (<ipython-input-59-3abbedc0f56d>, line 1)